# Train Models

In [7]:
import numpy as np

import os
from pathlib import Path

from nvita.utils import open_json
from nvita.utils import to_json
from nvita.models.data import SplittedTSData
from nvita.models.cnn import CNN
from nvita.models.lstm import LSTM
from nvita.models.gru import GRU
from nvita.models.rf import RF
from nvita.models.utils import save_model, load_model
import nvita.models.train as mt

In [8]:

"""
a_dict = dict()
a_dict["Electricity"] = {"CNN" : [500, 0.001, 4, 64, 3, 64, 64, 1], "LSTM" : [500, 0.001, 3, 64, 1, 1], "GRU": [500, 0.001, 3, 64, 1, 1], "RF":[500]}

to_json(a_dict, os.path.join(
    PATH_ROOT, "experiments", "model_paras.json"))
"""

'\na_dict = dict()\na_dict["Electricity"] = {"CNN" : [500, 0.001, 4, 64, 3, 64, 64, 1], "LSTM" : [500, 0.001, 3, 64, 1, 1], "GRU": [500, 0.001, 3, 64, 1, 1], "RF":[500]}\n\nto_json(a_dict, os.path.join(\n    PATH_ROOT, "experiments", "model_paras.json"))\n'

In [9]:
PATH_ROOT = Path(os.getcwd()).parent.absolute()

my_metadata = open_json(os.path.join(
    PATH_ROOT, "experiments", "metadata.json"))

model_paras = open_json(os.path.join(
    PATH_ROOT, "experiments", "model_paras.json"))
print(model_paras)

{'Electricity': {'CNN': [500, 0.001, 4, 64, 3, 64, 64, 1], 'LSTM': [500, 0.001, 3, 64, 1, 1], 'GRU': [500, 0.001, 3, 64, 1, 1], 'RF': [500]}}


In [10]:
SEEDS = my_metadata["seeds"]
MODELS = my_metadata["models"]
DATA_NAMES =  my_metadata["data"]

In [11]:
def train_models_based_on_model_paras(model_paras_dict, s_d):
    for model_name in MODELS:
        if model_name != "RF":
            if model_name == "CNN":
                epoch, lr, window_size, c, f0, f1, f2, out = model_paras_dict[s_d.df_name][model_name]
                model = CNN(window_size, c, f0, f1, f2, out)
            elif model_name == "LSTM":
                epoch, lr, input_dim, hidden_dim, num_layers, output_dim = model_paras_dict[s_d.df_name][model_name]
                model = LSTM(input_dim, hidden_dim, num_layers, output_dim)
            elif model_name == "GRU":
                epoch, lr, input_dim, hidden_dim, num_layers, output_dim = model_paras_dict[s_d.df_name][model_name]
                model = GRU(input_dim, hidden_dim, num_layers, output_dim)
            mt.train(model, lr, epoch, s_d.X_train, s_d.y_train)
        else:
            # model_name == "RF"
            n_estimators = model_paras_dict[s_d.df_name][model_name][0]
            model = RF(n_estimators)
            model.fit(s_d.X_train, s_d.y_train)
        score = np.mean(mt.evaluate(model, s_d.X_test, s_d.y_test)**2)
        save_model(model, PATH_ROOT, s_d.df_name, s_d.seed)
        print("Best Score for " + model_name + " is " + str(score) + " on dataset " + str(s_d.df_name) + " with seed " + str(s_d.seed))
    

In [12]:
#for data_ind in range(len(DATA_NAMES)):
for data_ind in range(1):
    for seed in SEEDS:
        s_data = SplittedTSData()
        s_data = s_data.load_splitted_data(PATH_ROOT, DATA_NAMES[data_ind], seed)
        train_models_based_on_model_paras(model_paras, s_data)

Best Score for CNN is 0.030555123 on dataset Electricity with seed 2210
Best Score for LSTM is 0.029053124 on dataset Electricity with seed 2210
Best Score for GRU is 0.030408725 on dataset Electricity with seed 2210
Best Score for RF is 0.010595608996610556 on dataset Electricity with seed 2210
Best Score for CNN is 0.03050473 on dataset Electricity with seed 9999
Best Score for LSTM is 0.025970764 on dataset Electricity with seed 9999


KeyboardInterrupt: 

In [13]:

s_data = SplittedTSData()
s_data = s_data.load_splitted_data(PATH_ROOT, DATA_NAMES[0], SEEDS[0])
m = load_model(PATH_ROOT, s_data.df_name, SEEDS[0], "CNN")

In [ ]:
y_p = mt.predict(m, s_data.X_test)

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'